## Option 3 - Spread of Covid 19

Look at this document for reference,
https://www.kaggle.com/anandhuh/covid19-in-world-countrieslatest-data

In [1]:
# (1) Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Import the machine learning model (which we want to evaluate)
from sklearn.ensemble import RandomForestClassifier
# Import the function for evaluating the model
from sklearn.model_selection import cross_val_score

# Import the ml model
from sklearn.ensemble import RandomForestClassifier

# Import GridSearchCV for finding the model with the best parameters
from sklearn.model_selection import GridSearchCV

# Import functions for measuring metrics of the ml model
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
# (2) Import the CSV file
covid19Df = pd.read_csv('./worldwide covid data.csv')

In [3]:
# (3) Explore data (for example, see what is categorial and numerical)
covid19Df.head()

,Country,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Cases/1M population,Deaths/1M population,Total Tests,Tests/1M population,Population
0,Afghanistan,156307,7281,128791.0,20235.0,3898,182,774655.0,19319.0,40097200
1,Albania,186222,2937,175451.0,7834.0,64805,1022,1311540.0,456411.0,2873596
2,Algeria,206649,5927,141811.0,58911.0,4602,132,230861.0,5141.0,44907419
3,Andorra,15516,130,15242.0,144.0,200387,1679,193595.0,2500258.0,77430
4,Angola,64487,1713,53376.0,9398.0,1884,50,1092363.0,31915.0,34227629


In [4]:
covid19Df.describe()

,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Cases/1M population,Deaths/1M population,Total Tests,Tests/1M population,Population
count,1.960000e+02,196.000000,1.940000e+02,1.940000e+02,196.000000,196.000000,1.920000e+02,1.920000e+02,1.960000e+02
mean,1.266763e+06,25658.688776,1.150839e+06,9.260771e+04,56113.617347,914.311224,2.094107e+07,1.060358e+06,4.008576e+07
std,4.593222e+06,84846.606103,4.046744e+06,6.744275e+05,51997.542468,955.591341,7.512503e+07,1.876692e+06,1.482140e+08
min,2.621000e+03,3.000000,5.500000e+01,2.000000e+00,68.000000,3.000000,1.468900e+04,3.287000e+03,3.402600e+04
25%,2.237675e+04,333.500000,1.590175e+04,6.695000e+02,5636.000000,112.500000,3.540520e+05,1.051550e+05,1.987078e+06
50%,1.778875e+05,2434.500000,1.402300e+05,7.220000e+03,46790.000000,628.000000,1.978244e+06,4.632715e+05,8.907116e+06
75%,6.605052e+05,11778.500000,6.085508e+05,3.443275e+04,91629.750000,1560.750000,1.112483e+07,1.280447e+06,2.893874e+07
max,4.699977e+07,768847.000000,3.700999e+07,9.220934e+06,231328.000000,5964.000000,7.034355e+08,1.481471e+07,1.439324e+09


In [5]:
featuresDf = covid19Df
labels = []

for index, row in featuresDf.iterrows():
    score = row['Deaths/1M population']/row['Total Cases/1M population']*100
    if score < 2.5:
        labels.append(False)
    else: 
        labels.append(True)

print(len(labels))
featuresDf['High/ Low Death Rate'] = pd.DataFrame( data=labels, columns=['High/ Low Death Rate'] )


196


In [6]:
# (4) Choose the label and features
# We will chose the label and features after cleaning the data 

#featuresDf = covid19Df

In [7]:
# (5) Feature engineer for data that is
# (a) unique
# Prove No Duplicates
print("Before Dropping Duplicates : ",featuresDf.shape,'\n')
featuresDf =featuresDf.drop_duplicates()
print("\nAfter Dropping Duplicates : ",featuresDf.shape)

Before Dropping Duplicates :  (196, 11) 


After Dropping Duplicates :  (196, 11)


In [8]:
# (5) Feature engineer for data that is
# (b) correct 
#Make sure no value is negative 
#Finding Negative Values and Capping To 0.
for col in featuresDf.columns:
    if(col=='Country'):
        continue
    for j in range(len(featuresDf)):
        if(featuresDf[col][j]<0):
            featuresDf[col][j]=0

In [9]:
# (5) Feature engineer for data that is
# (c) not missing
# First check the number of NULLS 
featuresDf.isna().sum()

Country                      0
Total Cases                  0
Total Deaths                 0
Total Recovered              2
Active Cases                 2
Total Cases/1M population    0
Deaths/1M population         0
Total Tests                  4
Tests/1M population          4
Population                   0
High/ Low Death Rate         0
dtype: int64

In [10]:
# Drop any rows with missing data 
featuresDf.dropna(inplace=True)

In [11]:
# Check the data types 
featuresDf.head()

,Country,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Cases/1M population,Deaths/1M population,Total Tests,Tests/1M population,Population,High/ Low Death Rate
0,Afghanistan,156307,7281,128791.0,20235.0,3898,182,774655.0,19319.0,40097200,True
1,Albania,186222,2937,175451.0,7834.0,64805,1022,1311540.0,456411.0,2873596,False
2,Algeria,206649,5927,141811.0,58911.0,4602,132,230861.0,5141.0,44907419,True
3,Andorra,15516,130,15242.0,144.0,200387,1679,193595.0,2500258.0,77430,False
4,Angola,64487,1713,53376.0,9398.0,1884,50,1092363.0,31915.0,34227629,True


In [12]:
# Covert float64 into int64
featuresDf['Tests/1M population'] = featuresDf['Tests/1M population'].apply(np.int64)

featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191 entries, 0 to 195
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Country                    191 non-null    object 
 1   Total Cases                191 non-null    int64  
 2   Total Deaths               191 non-null    int64  
 3   Total Recovered            191 non-null    float64
 4   Active Cases               191 non-null    float64
 5   Total Cases/1M population  191 non-null    int64  
 6   Deaths/1M population       191 non-null    int64  
 7   Total Tests                191 non-null    float64
 8   Tests/1M population        191 non-null    int64  
 9   Population                 191 non-null    int64  
 10  High/ Low Death Rate       191 non-null    bool   
dtypes: bool(1), float64(3), int64(6), object(1)
memory usage: 16.6+ KB


### Adding Labels (High Death Rate Countries Vs. Low Death Rate Countries) 
For the purpose of this project, we decided that we will train our model to classify countries into High Death Rate Country Vs. Low Death Rate Country. The meathod we did to create the labeling is finiding the mean for the death rate accross all the countries. 

In [13]:
featuresDf

,Country,Total Cases,Total Deaths,Total Recovered,Active Cases,Total Cases/1M population,Deaths/1M population,Total Tests,Tests/1M population,Population,High/ Low Death Rate
0,Afghanistan,156307,7281,128791.0,20235.0,3898,182,774655.0,19319,40097200,True
1,Albania,186222,2937,175451.0,7834.0,64805,1022,1311540.0,456411,2873596,False
2,Algeria,206649,5927,141811.0,58911.0,4602,132,230861.0,5141,44907419,True
3,Andorra,15516,130,15242.0,144.0,200387,1679,193595.0,2500258,77430,False
4,Angola,64487,1713,53376.0,9398.0,1884,50,1092363.0,31915,34227629,True
...,...,...,...,...,...,...,...,...,...,...,...
191,Venezuela,408727,4915,393040.0,10772.0,14428,174,3359014.0,118575,28328211,False
192,Vietnam,932357,22205,824806.0,85346.0,9465,225,60535102.0,614517,98508358,False
193,Yemen,9806,1894,6458.0,1454.0,319,62,265253.0,8640,30701818,True
194,Zambia,209760,3661,205984.0,115.0,11000,192,2606571.0,136686,19069835,False


In [14]:
# # (5) Feature engineer for data that is
# # (a) relevant
# # Droping irrelevant features 
featuresDf = featuresDf.drop(['Total Cases','Total Deaths','Total Recovered',
                              'Total Tests', 'Active Cases', 'Country'], axis=1,inplace =False)


In [15]:
# (6) Confirm data is ready with further exploratory analysis



# sns.scatterplot( 
#     data=featuresDf, 
#     x='Total Cases/1M population',
#     y='Deaths/1M population', 
#     hue='Tests/1M population'
# )

LabelDf = featuresDf['High/ Low Death Rate']
featuresDf = featuresDf.drop('High/ Low Death Rate', axis=1,inplace =False)
print(featuresDf.shape)

print(LabelDf.shape)

(191, 4)
(191,)


In [16]:
# (7) Training, Testing (and/or Validation) data split 

# for example, 60/20/20

# Training we have 60% and 40% for test
trainF, testF, trainL, testL = train_test_split( featuresDf, LabelDf, test_size=0.4, random_state=42 )

# Divide the test into two parts: 50% for test and 50% for validation
testF, valF, testL, valL = train_test_split( testF, testL, test_size=0.5, random_state=42 )

In [17]:
#Cross Validation 
# Instantiate the RandomForestClassifier
rfModel = RandomForestClassifier()
scores = cross_val_score( rfModel, trainF, trainL.values.ravel(), cv=5 )
print( scores )

[0.82608696 0.82608696 0.82608696 0.82608696 0.72727273]


In [18]:
def printResults(gridSearchResults):

    print( 'Best ML Hyperparameters: {}\n'.format(gridSearchResults.best_params_) )

    means = gridSearchResults.cv_results_['mean_test_score']
    stnDvs = gridSearchResults.cv_results_['std_test_score']

    for mean, stnDvs, parameters in zip( means, stnDvs, gridSearchResults.cv_results_['params'] ):
        print(
            '{} (+/-{}) for {}'.format(
                round(mean, 3),
                round(stnDvs*2, 3),
                parameters
            ) 
        )

In [19]:
# Instantiate the model and set the hyperparameters 
rfModel = RandomForestClassifier()

hyperparameters = {
    'n_estimators': [5, 50, 100],
    'max_depth': [2, 10, 20, None]
}

# GridSearchCV will search and rank the models according their parameters
# Note:
# sklearn refers to 'hyperparameters' as 'parameters'
gridSearch = GridSearchCV( rfModel, hyperparameters, cv=5 )

# .fit() will have the model, rfModel, learn the parameters of the features
gridSearch.fit( trainF, trainL.values.ravel() )

printResults( gridSearch )

Best ML Hyperparameters: {'max_depth': 10, 'n_estimators': 50}

0.676 (+/-0.082) for {'max_depth': 2, 'n_estimators': 5}
0.711 (+/-0.099) for {'max_depth': 2, 'n_estimators': 50}
0.737 (+/-0.078) for {'max_depth': 2, 'n_estimators': 100}
0.754 (+/-0.097) for {'max_depth': 10, 'n_estimators': 5}
0.806 (+/-0.096) for {'max_depth': 10, 'n_estimators': 50}
0.798 (+/-0.078) for {'max_depth': 10, 'n_estimators': 100}
0.763 (+/-0.107) for {'max_depth': 20, 'n_estimators': 5}
0.806 (+/-0.079) for {'max_depth': 20, 'n_estimators': 50}
0.789 (+/-0.091) for {'max_depth': 20, 'n_estimators': 100}
0.789 (+/-0.125) for {'max_depth': None, 'n_estimators': 5}
0.806 (+/-0.129) for {'max_depth': None, 'n_estimators': 50}
0.789 (+/-0.091) for {'max_depth': None, 'n_estimators': 100}


In [25]:
# (8) Training the Machine Learning Model (i.e, Fitting the Model)

rfModel1 = RandomForestClassifier(n_estimators=50, max_depth=None)
rfModel1.fit( trainF, trainL.values.ravel() )

rfModel2 = RandomForestClassifier(n_estimators=50, max_depth=20)
rfModel2.fit( trainF, trainL.values.ravel() )

rfModel3 = RandomForestClassifier(n_estimators=50, max_depth=10)
rfModel3.fit( trainF, trainL.values.ravel() )

RandomForestClassifier(max_depth=10, n_estimators=50)

In [26]:
# (9) Evaluate the model metrics for Training (and/or Validation) data
for rfML in [rfModel1, rfModel2, rfModel3]:

    predictedLabels = rfML.predict(valF)

    # Find the metrics for the model: (a) accuracy, (b) precision, and (c) recall
    accuracy = round(accuracy_score(valL, predictedLabels), 3)
    precision = round(precision_score(valL, predictedLabels), 3)
    recall = round(recall_score(valL, predictedLabels), 3)

    # Print the metrics for each model
    print(
        'Max depth: {} and Estimators: {}'.format(rfML.max_depth, rfML.n_estimators)
    )
    print(
        'Accuracy: {}, Precision: {}, Recall: {}'.format( accuracy, precision, recall )
    )
    print('-------------------------------------------------')

Max depth: None and Estimators: 50
Accuracy: 0.846, Precision: 0.625, Recall: 0.625
-------------------------------------------------
Max depth: 20 and Estimators: 50
Accuracy: 0.872, Precision: 0.714, Recall: 0.625
-------------------------------------------------
Max depth: 10 and Estimators: 50
Accuracy: 0.872, Precision: 0.714, Recall: 0.625
-------------------------------------------------


In [27]:
# We decided to use the third model becuase it has the closest recall and percision
# (10) Evaluate the model metris for Testing data
predictedLabels = rfModel2.predict( testF )

# Find the metrics for the model: (a) accuracy, (b) precision, and (c) recall
accuracy = round(accuracy_score(testL, predictedLabels), 3)
precision = round(precision_score(testL, predictedLabels), 3)
recall = round(recall_score(testL, predictedLabels), 3)

# Print the metrics for each model
print(
    'Max depth: {} and Estimators: {}'.format(rfML.max_depth, rfML.n_estimators)
)
print(
    'Accuracy: {}, Precision: {}, Recall: {}'.format( accuracy, precision, recall )
)
print('-------------------------------------------------')

Max depth: 10 and Estimators: 50
Accuracy: 0.842, Precision: 0.833, Recall: 0.5
-------------------------------------------------


In [28]:
featuresDf.head()

,Total Cases/1M population,Deaths/1M population,Tests/1M population,Population
0,3898,182,19319,40097200
1,64805,1022,456411,2873596
2,4602,132,5141,44907419
3,200387,1679,2500258,77430
4,1884,50,31915,34227629


In [30]:
# (11) Use the model for prediction
newDataFeatures = {
    'Total Cases/1M population': [800,1000],
    'Deaths/1M population': [5, 500],
    'Tests/1M population': [10,50],
    'Population': [3000, 4000],
}

newDataFeaturesDf = pd.DataFrame(data=newDataFeatures)
newDataFeaturesDf.head()

,Total Cases/1M population,Deaths/1M population,Tests/1M population,Population
0,800,5,10,3000
1,1000,500,50,4000


In [31]:
predictedLabels = rfModel2.predict( newDataFeaturesDf )
newDataFeaturesDf['High/ Low Death Rate'] = predictedLabels
newDataFeaturesDf.head()

,Total Cases/1M population,Deaths/1M population,Tests/1M population,Population,High/ Low Death Rate
0,800,5,10,3000,False
1,1000,500,50,4000,True


In [24]:
# (12) Write final predicted data  (e.g, to CSV or JSON, etc.)